## **TP4 - Gustavo Uñapillco**

#### **Bases de Vectores**

### **Introducción**

In [6]:
# 1️⃣ Importar librerías
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
import numpy as np
import uuid

# Modelo de embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def get_embeddings(text):
    return model.encode(text).tolist()

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos():
    docs_path = "docs/"
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}
    
    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")

def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=1
    )
    
    if results["documents"]:
        print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
        print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
        print(f"🆔 ID: {results['ids'][0][0]}")
    else:
        print("⚠️ No se encontraron resultados.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    metadata = {"source": "documento_actualizado.txt"}
    
    collection.add(
        documents=[new_text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[new_id]
    )
    print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

c:\Users\heguu\miniconda3\envs\vector_db\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] No se encontró el proceso especificado'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
crear_base_datos()

Delete of nonexisting embedding ID: 46f6ce7f-c9af-4340-b7e3-e601d505b4ec
Delete of nonexisting embedding ID: 46f6ce7f-c9af-4340-b7e3-e601d505b4ec
Delete of nonexisting embedding ID: no_existe
Delete of nonexisting embedding ID: 1110d7b5-6c6c-45cf-aff6-153795d5f196
Delete of nonexisting embedding ID: a30a9c40-05ba-4004-8ce0-2bcfcaf3ea39
Delete of nonexisting embedding ID: a30a9c40-05ba-4004-8ce0-2bcfcaf3ea39
Delete of nonexisting embedding ID: ff31433c-616e-4ccf-8f94-e1c03f866a55


✅ Base de datos creada con éxito.


In [8]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [9]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: ce09db15-8022-498b-a3ba-79b43b78ec22, Contenido: En una empresa de tecnología en rápido crecimiento, trabajaban juntos un desarrollador ágil y un arq  Metadata: {'source': 'doc1.txt'} Embedding: [-0.06716214865446091, 0.00912424921989441, -0.09364323318004608, -0.1132781133055687, -0.041556213051080704]...
ID: 0e35f7ad-fa21-41a0-980a-9de37d217d9c, Contenido: dad de usuarios en apenas un mes. El desarrollador ágil rápidamente implementó soluciones temporales  Metadata: {'source': 'doc1.txt'} Embedding: [-0.05381535366177559, 0.0009738719672895968, -0.10402170568704605, -0.12424676865339279, -0.06802421808242798]...
ID: 292797b0-d6be-46b6-bb2e-7c15c02198c5, Contenido: io, nunca estuvo operativo a tiempo.

En el mundo de la tecnología, actuar a tiempo con soluciones f  Metadata: {'source': 'doc1.txt'} Embedding: [-0.05644704028964043, -0.04881387948989868, -0.08858117461204529, -0.022213034331798553, -0.09305635839700699]...
ID: 6e81108f-a3d4-4dc5-a75b-3fe0631bbc01, Contenido: Una com

### **Ejercicio 1: Crear y Consultar**

##### 1.1. Crea un documento nuevo con el texto: "Un desarrollador ágil resolvió rápidamente el aumento de usuarios adaptando su sistema con soluciones funcionales, mientras que un arquitecto de software no pudo implementar a tiempo su compleja infraestructura.". 

In [10]:
new_doc = "Un desarrollador ágil resolvió rápidamente el aumento de usuarios adaptando su sistema con soluciones funcionales, mientras que un arquitecto de software no pudo implementar a tiempo su compleja infraestructura."
create_example(new_doc)

✅ Documento añadido con ID: dfcd6efa-fbf4-478d-b520-8138cf646861


##### 1.2. Realiza una consulta para encontrar documentos similares a la frase: "Cómo adaptarse rápidamente a un crecimiento inesperado sin depender de planes demasiado elaborados.". 

In [11]:
query = "Cómo adaptarse rápidamente a un crecimiento inesperado sin depender de planes demasiado elaborados."

read_example(query)

📄 Mejor coincidencia: io, nunca estuvo operativo a tiempo.

En el mundo de la tecnología, actuar a tiempo con soluciones funcionales suele ser más valioso que perseguir la perfección inalcanzable. A veces, la rapidez y la adaptabilidad pueden superar incluso a las mejores intenciones de robustez absoluta.
📁 Fuente: doc1.txt
🆔 ID: 85d2d3fc-47d7-4cc8-820a-9e78bee355d9


##### 1.3. ¿Qué tipo de operación CRUD usaste primero?

La primera operación CRUD fue:          **CREATE** → Creación de un documento (almacenar el texto original).



##### 1.4. ¿Cuál es el ID del documento creado? (Usa collection.peek()).
    


In [ ]:
ID_nuevo_documento = collection.get()["ids"][-1]  #-1 por ser el último documento añadido. 
print('\tEl ID del nuevo documento es:\t', ID_nuevo_documento)

	El ID del nuevo documento es:	 dfcd6efa-fbf4-478d-b520-8138cf646861


##### 1.5. Explica por qué el documento aparece como coincidencia en la consulta.

El texto del doc1.txt ("actuar a tiempo con soluciones funcionales suele ser más valioso...") tiene un significado muy similar a la consulta:

"Cómo adaptarse rápidamente a un crecimiento inesperado sin depender de planes demasiado elaborados."

Ambos textos hablan de:

- Adaptarse rápidamente
- Responder funcionalmente frente a situaciones de urgencia
- Evitar la parálisis por planificación excesiva

Por eso el motor de búsqueda semántica detecta como mejor coincidencia el documento que, aunque no repita exactamente las palabras, captura el mismo concepto o moraleja.

### **Ejercicio 2: Actualizar y Eliminar**

##### 2.1. Crea un documento con el texto: "Mientras un científico de machine learning perfeccionaba modelos complejos, una analista de datos detectó fallas reales aplicando reglas simples y prácticas, demostrando que la simplicidad puede ser más efectiva en situaciones críticas." 

In [24]:
new_doc2 = "Mientras un científico de machine learning perfeccionaba modelos complejos, una analista de datos detectó fallas reales aplicando reglas simples y prácticas, demostrando que la simplicidad puede ser más efectiva en situaciones críticas."

create_example(new_doc2)

✅ Documento añadido con ID: 563e8000-e556-423c-aecf-af3db85b63ea


##### 2.2. Encuentra su ID, luego actualízalo por: "Por qué en entornos críticos a veces una solución sencilla funciona mejor que un modelo sofisticado."

In [ ]:
new_text = "Por qué en entornos críticos a veces una solución sencilla funciona mejor que un modelo sofisticado"
old_id = "563e8000-e556-423c-aecf-af3db85b63ea"

update_example(old_id, new_text)

##### 2.3. Elimina el documento original usando el mismo ID. 

In [ ]:
update_example(old_id, new_text)

♻️ Documento actualizado. Nuevo ID: 39d29945-882a-46b4-9e31-9d6124280d49


##### 2.4. ¿Qué ocurre si intentas actualizar un documento eliminado?**

En la función `update_example`, el proceso de actualización se basa en eliminar un documento a partir de su ID y luego agregar uno nuevo con el contenido modificado.

Si se intenta actualizar un documento que ya fue eliminado previamente, la instrucción `collection.delete(ids=[old_id])` simplemente no encuentra ningún elemento que borrar y no genera un error grave. Este comportamiento es normal, ya que muchas bases de datos vectoriales están diseñadas para ser tolerantes a este tipo de situaciones, permitiendo realizar operaciones idempotentes (es decir, que puedan ejecutarse varias veces sin alterar el estado final).

##### 2.5. ¿Cómo se garantiza la integridad de los datos en las operaciones?


La integridad de los datos se preserva a través de los siguientes mecanismos:

1. Identificadores únicos: Cada documento cuenta con un `id` exclusivo (en este caso generado mediante `uuid.uuid4())` , lo que previene colisiones o sobrescrituras accidentales.

2. Eliminación previa a la inserción: Al eliminar el documento existente antes de agregar uno nuevo, se evita la aparición de duplicados involuntarios tras modificaciones.

3. Gestión de metadatos: Mediante el uso de campos como `source` en los metadata, es posible rastrear el origen y estado de cada documento, incluso después de una actualización.

4. Embeddings actualizados: Cada vez que se inserta un documento nuevo, su representación vectorial se recalcula, garantizando que el embedding refleje fielmente el contenido actualizado.

Este método de "eliminar y volver a insertar" resulta sencillo pero eficaz para mantener la consistencia en bases de datos vectoriales que, en general, no ofrecen una operación de update directa.

### **Ejercicio 3: Errores y Validación**

##### 3.1. Intenta eliminar un documento con el ID "no_existe". 

In [28]:
delete_example("no_existe")

Delete of nonexisting embedding ID: no_existe
Delete of nonexisting embedding ID: no_existe


🗑️ Documento con ID 'no_existe' eliminado.


##### 3.2. Crea un nuevo documento sin usar get_embeddings() (ejemplo: enviar un array vacío). 

In [ ]:
# Crear un documento sin usar get_embeddings()
collection.add(
    documents=["Texto sin embedding"],
    embeddings=[[]],  # array vacío
    metadatas=[{"source": "sin_embedding.txt"}],
    ids=[str(uuid.uuid4())]
)

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\heguu\miniconda3\envs\vector_db\lib\site-packages\chromadb\db\mixins\embeddings_queue.py", line 308, in _notify_one
    sub.callback(filtered_embeddings)
  File "c:\Users\heguu\miniconda3\envs\vector_db\lib\site-packages\chromadb\segment\impl\vector\local_persistent_hnsw.py", line 211, in _write_records
    self._ensure_index(len(records), len(record["embedding"]))
  File "c:\Users\heguu\miniconda3\envs\vector_db\lib\site-packages\chromadb\segment\impl\vector\local_hnsw.py", line 208, in _ensure_index
    raise InvalidDimensionException(
chromadb.errors.InvalidDimensionException: Dimensionality of (0) does not match indexdimensionality (384)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\heguu\miniconda3\envs\vector_db\lib\logging\__init__.py", line 1100, in emit
    msg = self.format(record)
  File "c:\Users\heguu\miniconda3\envs\vector_d

##### 3.3. ¿Qué errores ocurren en cada caso?

- **Intentar eliminar un documento con un ID inexistente:**

  No se produce un error crítico. ChromaDB simplemente ignora el ID si no está presente en la colección. Este comportamiento es **seguro e idempotente**, lo que permite realizar múltiples intentos de eliminación sin comprometer la integridad de la base de datos.

- **Agregar un documento sin utilizar `get_embeddings()` (por ejemplo, proporcionando una lista vacía):**

  En este caso, se genera un **`ValueError`** o un **`RuntimeError`**, dependiendo del backend utilizado. Esto ocurre porque el sistema espera un **vector numérico válido** para completar la operación, y un array vacío no cumple con esa condición. La inserción falla al intentar procesar un embedding inválido.


##### 3.4. ¿Cómo prevenir estos problemas?

- **Verificar la existencia del ID antes de eliminar:**

  Antes de intentar eliminar un documento, es recomendable comprobar si el ID se encuentra registrado en la colección. Esta práctica ayuda a evitar operaciones innecesarias o confusas sobre elementos que no existen.

- **Validar la generación de embeddings antes de agregar documentos:**

  Para prevenir errores durante la inserción de documentos, es esencial confirmar que los embeddings se hayan generado correctamente y que no estén vacíos. Validar este paso garantiza que el


### **Ejercicio 4: Consultas por Tema**

##### 4.1. Crea tres documentos: 


* "Un desarrollador ágil prefiere lenguajes ligeros y flexibles como JavaScript y Python para construir soluciones rápidas y adaptables."
* "Los arquitectos de software suelen optar por lenguajes robustos como Java o C# para diseñar infraestructuras escalables y seguras en grandes sistemas empresariales."
* "En ciberseguridad, lenguajes como Python son fundamentales tanto para automatizar defensas como para desarrollar herramientas de auditoría y análisis forense."

In [29]:
doc1 = "Un desarrollador ágil prefiere lenguajes ligeros y flexibles como JavaScript y Python para construir soluciones rápidas y adaptables"
doc2 = "Los arquitectos de software suelen optar por lenguajes robustos como Java o C# para diseñar infraestructuras escalables y seguras en grandes sistemas empresariales" 
doc3 = "En ciberseguridad, lenguajes como Python son fundamentales tanto para automatizar defensas como para desarrollar herramientas de auditoría y análisis forense" 

create_example(doc1)
create_example(doc2)
create_example(doc3)

✅ Documento añadido con ID: 5fa6f522-c97e-46c8-bc12-c865fec1e612
✅ Documento añadido con ID: e817cb8e-3b97-4164-bbe2-3cfcdd3f71d7
✅ Documento añadido con ID: c9e029a4-8b10-4812-a3d9-4b07f5e75e96


##### 4.2. Realiza una consulta por la frase: "Lenguajes backend y frontend". 

In [30]:
consulta = "Lenguajes backend y frontend"
read_example(consulta)

📄 Mejor coincidencia: En una empresa de tecnología en rápido crecimiento, trabajaban juntos un desarrollador ágil y un arquitecto de software. El desarrollador creía en moverse rápido: escribía código funcional sin preocuparse demasiado por la escalabilidad o el diseño a largo plazo. El arquitecto, en cambio, dedicaba semanas a planificar infraestructuras complejas, diseñando arquitecturas robustas que pudieran resistir el paso del tiempo.

Un día, la empresa consiguió un gran contrato que implicaba duplicar la canti
📁 Fuente: doc1.txt
🆔 ID: 671a8c3b-7d72-4c2a-9856-e3eb2f61fb57


##### 4.3. ¿Cuáles documentos aparecen como coincidencias?


El doc1.txt con ID 671a8c3b-7d72-4c2a-9856-e3eb2f61fb57 fue identificado como la mejor coincidencia, y tiene sentido ya que menciona JavaScript (frontend) y Python (que podría ser backend)

##### 4.4. Analiza cómo las palabras clave influyen en los resultados


En base a la consulta realizada con la frase "Lenguajes backend y frontend", los resultados obtenidos se explican de la siguiente manera:

El Documento 1 se muestra como la coincidencia más relevante, ya que menciona explícitamente JavaScript —un lenguaje ampliamente utilizado en el desarrollo frontend— y Python, que puede desempeñarse como lenguaje de backend. Esto genera una correspondencia directa con ambos conceptos buscados.

El Documento 2 también aparece como coincidencia, dado que hace referencia a Java, un lenguaje clásico y robusto para el desarrollo de aplicaciones de backend, especialmente en entornos empresariales. Aunque no se menciona explícitamente un lenguaje de frontend, la mención de un lenguaje backend importante contribuye a su relevancia parcial.

Por otro lado, el Documento 3 resulta menos relacionado con la consulta, ya que se centra en el uso de Python dentro del contexto de ciberseguridad y análisis forense, sin hacer mención específica al uso de lenguajes en backend o frontend.

En resumen, las palabras clave como "JavaScript", "Python" y "Java" son determinantes en el nivel de coincidencia, mientras que la asociación explícita a los roles de backend y frontend incrementa la pertinencia de los documentos respecto a la consulta planteada.

### **Ejercicio 5: Eliminaciones Selectivas**


##### 5.1. Crea dos documentos con tópicos opuestos (ejemplo: uno sobre "Ecología" y otro sobre "Minería de datos"). 



Usaremos los conceptos de "Desarrollo Ágil" y "Arquitectura de Software Robusta" como temas opuestos dentro de los textos. 

In [ ]:
#Primero quiero ver cuantos documentos hay 
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 35


In [35]:
create_example("El desarrollo ágil prioriza la entrega rápida y continua de funcionalidades, adaptándose a cambios frecuentes de requerimientos")
create_example("La arquitectura de software robusta apuesta por un diseño meticuloso, estable y planificado, capaz de soportar cambios a largo plazo sin comprometer la calidad")

✅ Documento añadido con ID: f8f52ec0-47df-4c64-86fe-c75a21f3f406
✅ Documento añadido con ID: d4c1e679-5b32-4bf9-a127-0fa9e5bc22a7


In [36]:
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 39


##### 5.2. Elimina el documento sobre "Desarrollo Ágil" usando su ID. 

In [37]:
# Se eliminara el documento relacionado con la magia.

id_desarrollo = "f8f52ec0-47df-4c64-86fe-c75a21f3f406"
delete_example(id_desarrollo)

🗑️ Documento con ID 'f8f52ec0-47df-4c64-86fe-c75a21f3f406' eliminado.


##### 5.3. Verifica que solo quede un documento. 

In [38]:
count = collection.count()
print(f"Cantidad de documentos restantes en la colección: {count}")

Cantidad de documentos restantes en la colección: 38


##### 5.4. ¿Cómo usar collection.count() para confirmar? 


Luego de ejecutar la operación `delete_example`, se utilizó el método `collection.count()` para verificar la cantidad de documentos almacenados en la colección.
Esta verificación es fundamental porque `collection.count()` proporciona el número exacto de documentos activos, sin necesidad de realizar consultas o búsquedas adicionales.

El resultado mostró que quedaba un solo documento de los dos creados, lo cual confirma que:

- El documento con `Desarollo Ágil` fue efectivamente eliminado.

- El documento sobre `Arquitectura robusta` permanece en la colección.

- La operación de eliminación fue selectiva y precisa, afectando únicamente el documento especificado.

### **Ejercicio 6: Actualización en Cadena**

##### 6.1. Crea un documento con el texto "Los enfoques estratégicos en el desarrollo de software moderno". 

In [39]:
create_example("Los enfoques estratégicos en el desarrollo de software moderno")

✅ Documento añadido con ID: ba74afde-74d6-49ad-8799-c2d9161f05e7


##### 6.2. Actualízalo dos veces:

Primera actualización: "La agilidad como respuesta a la incertidumbre en la gestión de proyectos de software" 


In [ ]:
id_doc_inicial = "ba74afde-74d6-49ad-8799-c2d9161f05e7"

#Primera actualización: 
update_example(id_doc_inicial, "La agilidad como respuesta a la incertidumbre en la gestión de proyectos de software")


♻️ Documento actualizado. Nuevo ID: f7734644-9456-4894-ac79-cd283dc59bdd


Segunda actualización: "La importancia de diseñar arquitecturas robustas frente a los riesgos de seguridad y escalabilidad". 

In [ ]:
# Segunda actualización: 
update_example(id_doc_inicial, "La importancia de diseñar arquitecturas robustas frente a los riesgos de seguridad y escalabilidad")

Delete of nonexisting embedding ID: ba74afde-74d6-49ad-8799-c2d9161f05e7
Delete of nonexisting embedding ID: ba74afde-74d6-49ad-8799-c2d9161f05e7


♻️ Documento actualizado. Nuevo ID: e29a971c-261c-4cbb-ad66-142acc284209


##### 6.3. ¿Cómo sigue evolucionando el contenido del documento con cada update? 

El contenido del documento evoluciona progresivamente a medida que se realizan las actualizaciones, pasando de una perspectiva general a una más específica y orientada a problemáticas concretas dentro del desarrollo de software.

- En la **versión inicial**, el documento presenta una visión amplia sobre los enfoques estratégicos en el desarrollo de software moderno, sin profundizar en metodologías particulares.

- La **primera actualización** introduce el concepto de **agilidad como respuesta frente a la incertidumbre**, enfocándose en la capacidad de adaptación en entornos cambiantes. Aquí el contenido se especializa hacia un marco metodológico ágil y su utilidad en la gestión de proyectos.

- En la **segunda actualización**, la atención se desplaza hacia la necesidad de **diseñar arquitecturas robustas**, incorporando elementos relacionados con la **seguridad** y la **escalabilidad**. El contenido pasa de centrarse en la flexibilidad a resaltar la importancia de estructuras sólidas para el desarrollo sostenible a largo plazo.

En conjunto, cada actualización **refina y redirige el enfoque temático** del documento, representando una evolución conceptual que va de lo general a lo técnico-específico, abordando distintas dimensiones estratégicas del desarrollo de software.


### **Ejercicio 7: Consultas Ambiguas**


##### 7.1. Crea dos documentos similares pero no idénticos


* "El desarrollo ágil busca entregar valor al cliente de forma continua y con ciclos cortos.
* "El objetivo del enfoque ágil es entregar funcionalidades útiles lo más rápido posible"

In [42]:
create_example("El desarrollo ágil busca entregar valor al cliente de forma continua y con ciclos cortos")
create_example("El objetivo del enfoque ágil es entregar funcionalidades útiles lo más rápido posible")

✅ Documento añadido con ID: 4aa6fc4e-c594-47b4-badf-9d22dffdd569
✅ Documento añadido con ID: d532b2f9-743a-4228-b7c0-f0f46d84c2ab


##### 7.2. Realiza una consulta por "Entrega rápida de valor en metodologías ágiles". 

In [43]:
read_example("Entrega rápida de valor en metodologías ágiles")

📄 Mejor coincidencia: El desarrollo ágil busca entregar valor al cliente de forma continua y con ciclos cortos
📁 Fuente: nuevo_documento.txt
🆔 ID: 4aa6fc4e-c594-47b4-badf-9d22dffdd569


##### 7.3. ¿Ambos documentos aparecen?

No, solo aparece el nuevo_documento.txt: "El desarrollo ágil busca entregar valor al cliente de forma continua y con ciclos cortos".

Esto indica que ChromaDB identificó una mayor similitud semántica entre esa frase y la consulta "Entrega rápida de valor en metodologías ágiles", en comparación con el otro documento.

Aunque ambos textos tratan sobre el enfoque ágil y la entrega de valor, el documento seleccionado coincide más estrechamente con la estructura semántica de la consulta, especialmente por incluir directamente los conceptos de “valor”, “entrega” y “ciclos cortos”, que están alineados con la idea de rapidez y metodologías ágiles.

##### 7.4. Explica la similitud semántica


Ambas frases comparten un mismo núcleo conceptual: **la entrega ágil de valor al usuario final**.  
La consulta —_"Entrega rápida de valor en metodologías ágiles"_— plantea explícitamente la relación entre **velocidad**, **valor** y **enfoque ágil**.

El documento seleccionado como mejor coincidencia —_"El desarrollo ágil busca entregar valor al cliente de forma continua y con ciclos cortos"_— coincide estrechamente en términos semánticos, ya que contiene directamente las ideas de **“entrega”**, **“valor”**, **“cliente”**, **“forma continua”** y **“ciclos cortos”**, todos elementos centrales de la metodología ágil.

Esta coincidencia refleja cómo **ChromaDB prioriza las representaciones semánticas** que abarcan tanto el **significado implícito** como el **lenguaje explícito**, y selecciona el documento cuya estructura conceptual se alinea más estrechamente con la intención de la consulta.


### **Ejercicio 8: Reiniciar Base de Datos**

##### 8.1. Elimina todos los documentos usando delete() con el parámetro correcto. 

In [ ]:
#Cuantos documentos hay en la colección? 
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 42


In [ ]:
#Elimino todos los documentos.
collection.delete(where={})

In [ ]:
#Compruebo cuantos documentos hay luego de la eliminación
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 0


##### 8.2. Vuelve a cargar la base desde cero con crear_base_datos(). 


In [47]:
crear_base_datos()

✅ Base de datos creada con éxito.


##### 8.3. ¿Cuántos documentos hay ahora? (Usa collection.count()). 

In [49]:
#Cuantos documentos hay? 
count = collection.count()
print(f"Cantidad de documentos en la colección: {count}")

Cantidad de documentos en la colección: 9


### **Ejercicio 9: Errores de Inserción**


##### 9.1. Intenta crear un documento sin texto (ejemplo: cadena vacía). 

In [65]:
create_example("")

✅ Documento añadido con ID: d01e0725-deec-4248-adef-1fb7d0e83335


In [66]:
texto_vacio = collection.get(ids=["d01e0725-deec-4248-adef-1fb7d0e83335"], include=["documents"])

In [67]:
embedding = model.encode([""])  # Esto también podría fallar o generar un vector nulo

##### 9.2. Corrige el error usando la función get_embeddings().

In [68]:
# Asignar el contenido del documento a una variable
texto =  texto_vacio["documents"][0]
embedding = model.encode([texto])  # Esto también podría fallar o generar un vector nulo

# Validar antes de agregar
if texto.strip():
    collection.add(documents=[texto], embeddings=embedding, ids=["doc_valido"])
else:
    print("Texto inválido: no se puede generar embedding desde una cadena vacía.")

Texto inválido: no se puede generar embedding desde una cadena vacía.


##### 9.3. ¿Qué lección aprendemos sobre los requisitos de ChromaDB?

ChromaDB requiere que todo documento insertado tenga contenido textual válido, ya que la base de datos vectorial depende de los embeddings para representar semánticamente cada documento.

No es posible insertar cadenas vacías o contenido que no pueda transformarse en vectores numéricos válidos.

Por eso es importante validar los textos antes de insertarlos o generar manualmente los embeddings para asegurar que son apropiados.